# Introduction

The Taxi Problem
from "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"
by Tom Dietterich

helpful solution: 
[https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/)

[https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py)

[https://github.com/openai/gym/blob/master/gym/core.py](https://github.com/openai/gym/blob/master/gym/core.py)

## Description:
There are four designated locations in the grid world indicated by R(ed), B(lue), G(reen), and Y(ellow). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drive to the passenger's location, pick up the passenger, drive to the passenger's destination (another one of the four specified locations), and then drop off the passenger. Once the passenger is dropped off, the episode ends.<p>

Observations: 
There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is the taxi), and 4 destination locations. 
    
## Actions: 
There are 6 discrete deterministic actions:
- 0: move south
- 1: move north
- 2: move east 
- 3: move west 
- 4: pickup passenger
- 5: dropoff passenger
    
## Rewards: 
There is a reward of -1 for each action and an additional reward of +20 for delievering the passenger. There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.
    
## Rendering:
- blue: passenger
- magenta: destination
- yellow: empty taxi
- green: full taxi
- other letters: locations

In [63]:
import gym
from gym import envs
import numpy as np

env = gym.make("Taxi-v2").env #env = gym.make('Taxi-v2')
env.reset()

#print(envs.registry.all())

29

In [ ]:
# in the grid, the upper left corner is 0,0 
# the lower left corner is 4,0
# we then count in the style (y,x)

In [ ]:
# State / Observation space
env.observation_space

In [ ]:
#show reward table for all 500 possible states
# Agent learns from this. 

# taxi locations: 5*5 squares = 25
# passenger locations: 4 + 1(in taxi) = 5
# destination locations: 4
# 5×5×5×4=500 total possible states.

#Example: 
# One state could be: Taxi is in location 3/0 , Passenger is in Location G, Destination is in location B
# Another state could be identical except Taxi is in location 3/1

#each row is a state, columns are probability, next state, reward, done
env.P

In [64]:
env.action_space

Discrete(6)

In [65]:
'''When the Taxi environment is created, there is an initial 
Reward table that's also created, called `P`. We can think of it like a matrix 
that has the number of states as rows and number of actions as columns, 
i.e. a states × actions matrix.

Dictionary has structure {action: [(probability, nextstate, reward, done)]}
'''

'''There are 6 discrete deterministic actions:
- 0: move south
- 1: move north
- 2: move east 
- 3: move west 
- 4: pickup passenger
- 5: dropoff passenger'''

env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

## Testing the setup

In [61]:
# Set a random state and render
state, reward, done, info = env.step(env.action_space.sample())
env.render()

#state = env.reset()
#env.render()

# Set the state by encoding all positions
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

# Set env.s and render (do we also have to reset here?)
env.s = 328
env.render()

+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


## Bruteforce method, no learning

In [ ]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.01)
        
print_frames(frames)

## Using Reinforcement Learning
In our Taxi environment, we have the reward table, P, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.<p>

The values store in the Q-table are called a Q-values, and they map to a (state, action) combination.<p>

A Q-value for a particular state-action combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards.<p>
    
For example, if the taxi is faced with a state that includes a passenger at its current location, it is highly likely that the Q-value for pickup is higher when compared to other actions, like dropoff or north.<p>
    
The Q-value of a state-action pair is the sum of the instant reward and the discounted future reward (of the resulting state).<p>
    
We store Q-Values in a Q-Table. The Q-table is a matrix where we have a row for every state (500) and a column for every action (6). It's first initialized to 0, and then values are updated after training. Note that the Q-table has the same dimensions as the reward table, but it has a completely different purpose. <p>
    

### Q-Table
Rows: States<br>
Columns: 
South(0)
North (1)
East (2)
West (3)
Pickup (4)
Dropoff (5)

In [62]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [46]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 35.3 s, sys: 6.75 s, total: 42.1 s
Wall time: 36.6 s


In [4]:
q_table[328]

array([-2.29463633, -1.97092096, -2.29906657, -2.20365701, -9.04558427,
       -8.83460355])

In [47]:
state = env.reset()
print(state)

331


In [48]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.66
Average penalties per episode: 0.0


## Record actions using new q-table

In [59]:
#state = 477  # set environment to illustration's state
state = env.reset()
#env.s = 228

penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 75
Penalties incurred: 0


In [60]:
## Show actions based on q-table        
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.5)
        
print_frames(frames)


## How to print values of each cell?

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 9
State: 16
Action: 5
Reward: 20
